In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors

# Load the dataset
github_url = 'https://github.com/ficiliatm/foodresQ/raw/main/Location_Dataset.xlsx'
df = pd.read_excel(github_url, engine='openpyxl')

X = df[['Latitude', 'Longitude']].values

# Standardize the input data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(2,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='linear')  # Output layer with 2 neurons for latitude and longitude
])

model.compile(optimizer='adam', loss='mse', metrics='accuracy')  # Using Mean Squared Error as loss for regression

# Split the data into training and testing sets
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

# Train the model
model.fit(X_train, X_train, epochs=10, batch_size=32, validation_split=0.1, verbose=1)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(X_test, predictions)
print(f'Mean Squared Error on Test Set: {mse}')

# Use Nearest Neighbors to find the nearest locations
neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(X_scaled)

# Test 10 nearest neighbors
query_location = np.array([[-6.8798526, 107.6114894]])
query_location_scaled = scaler.transform(query_location)
distances, indices = neigh.kneighbors(query_location_scaled)

# Print the nearest locations
print(f"Nearest locations to ({-6.8798526}, {107.6114894}):")
for index in indices[0]:
    print(f"Resto Name : {df.iloc[index]['Place_name']} | Latitude: {df.iloc[index]['Latitude']}, Longitude: {df.iloc[index]['Longitude']}")

Epoch 1/10
4/4 [==============================] - 3s 264ms/step - loss: 0.9028 - accuracy: 0.5327 - val_loss: 1.0167 - val_accuracy: 0.7500
Epoch 2/10
4/4 [==============================] - 0s 40ms/step - loss: 0.7161 - accuracy: 0.8411 - val_loss: 0.7988 - val_accuracy: 0.9167
Epoch 3/10
4/4 [==============================] - 0s 48ms/step - loss: 0.5599 - accuracy: 0.9813 - val_loss: 0.6177 - val_accuracy: 1.0000
Epoch 4/10
4/4 [==============================] - 0s 42ms/step - loss: 0.4240 - accuracy: 0.9907 - val_loss: 0.4657 - val_accuracy: 1.0000
Epoch 5/10
4/4 [==============================] - 0s 46ms/step - loss: 0.3121 - accuracy: 1.0000 - val_loss: 0.3392 - val_accuracy: 1.0000
Epoch 6/10
4/4 [==============================] - 0s 32ms/step - loss: 0.2213 - accuracy: 1.0000 - val_loss: 0.2332 - val_accuracy: 1.0000
Epoch 7/10
4/4 [==============================] - 0s 63ms/step - loss: 0.1512 - accuracy: 1.0000 - val_loss: 0.1487 - val_accuracy: 1.0000
Epoch 8/10
4/4 [==========

In [21]:
model.save('foodresq_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
